# 🥔 Patates Hastalık Sınıflandırma - ImageDataGenerator ile

Bu notebook, ImageDataGenerator kullanarak patates hastalık tespiti yapar.
ImageDataGenerator, gerçek zamanlı veri artırma sağlar ve bellek verimlidir.

**Geliştirici:** Mustafa  
**Tarih:** 2026

## 1. Gerekli Kütüphaneler

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

## 2. Sabit Değerler

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 50
data_dir = "../PlantVillage"

## 3. ImageDataGenerator ile Veri Hazırlama

ImageDataGenerator, veri artırma işlemlerini gerçek zamanlı olarak uygular:
- Görüntü döndürme
- Yatay/dikey çevirme
- Yakınlaştırma
- Parlaklık değişimi

In [ ]:
# Eğitim verisi için veri artırma
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2  # %20 doğrulama verisi
)

# Test verisi için sadece normalize et
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Eğitim verisi generator
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training'
)

# Doğrulama verisi generator
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation'
)

In [ ]:
# Sınıf isimlerini al
class_names = list(train_generator.class_indices.keys())
n_classes = len(class_names)
print(f"Sınıflar: {class_names}")

## 4. Örnek Görüntülerin Görselleştirilmesi

In [ ]:
images, labels = next(train_generator)

plt.figure(figsize=(12, 12))
for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(images[i])
    plt.title(class_names[int(labels[i])], fontsize=10)
    plt.axis("off")
plt.suptitle("Veri Artırma Uygulanmış Görüntüler", fontsize=14)
plt.tight_layout()

## 5. CNN Model Oluşturma

In [ ]:
model = models.Sequential([
    # Giriş katmanı
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Tam bağlantılı katmanlar
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # Overfitting önleme
    layers.Dense(n_classes, activation='softmax'),
])

model.summary()

## 6. Model Derleme ve Eğitim

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

## 7. Eğitim Grafiklerinin Çizilmesi

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Eğitim Doğruluğu', color='#667eea', linewidth=2)
plt.plot(epochs_range, val_acc, label='Doğrulama Doğruluğu', color='#f5576c', linewidth=2)
plt.legend(loc='lower right')
plt.title('Eğitim ve Doğrulama Doğruluğu', fontsize=12)
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Eğitim Kaybı', color='#667eea', linewidth=2)
plt.plot(epochs_range, val_loss, label='Doğrulama Kaybı', color='#f5576c', linewidth=2)
plt.legend(loc='upper right')
plt.title('Eğitim ve Doğrulama Kaybı', fontsize=12)
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Modelin Kaydedilmesi

In [ ]:
model.save('../saved_models/imagedatagen_model')
print("Model '../saved_models/imagedatagen_model' olarak kaydedildi.")